# Wealth dynamics

In this notebook, we will introduce the concept of wealth dynamics. Wealth dynamics determine **how wealth evolves over time**. We will consider the wealth process $x(t)$ that we will use to create gambles chosen by subjects of the experiment. Wealth will change over time, following a discrete process repeating every $\Delta t$ seconds. We can describe a single step of evolution as $x(t+\Delta t)=x(t)+D$, where $D$ is a random variable dependent on the dynamic.

In [ ]:
from utils.paralell import isoelastic_utility, wealth_change
from utils.style import rc_style, eta_dynamic_color

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

mpl.rcParams.update(rc_style)

## Isoelastic utility function

The construction of wealth dynamics will be based on the concept of the utility function. Utility function $u(x)$ is a mapping, $u:\mathbb{R}\rightarrow\mathbb{R}$, converting wealth into scalar value called **utility**. For all subsequent analysis, we will use [**Isoelastic utility function**](https://en.wikipedia.org/wiki/Isoelastic_utility), which is a family of functions parameterized by a single parameter called **risk-aversion parameter** and denoted as $\eta$:

$$u(x)=\frac{x^{1-\eta}-1}{1-\eta}$$

Below, you can play around with $\eta$ to see how it influence Isoelastic utility function. For $\eta < 0$, $u(x)$ is convex which leads to risk-seeking behaviors, whereas for $\eta > 0$, $u(x)$ is concave which results in agent's risk-aversion. For $\eta = 0$, $u(x)=x$ is an identy function resulting in risk-neutrality.

In [ ]:
def plot_isoelastic_function(eta, x_min=0.01, x_max=100):
    x = np.linspace(x_min, x_max, 100)
    y = isoelastic_utility(x, eta) 
    
    fig, ax = plt.subplots(figsize=(6, 6))

    ax.plot(x, y, label=f"$\eta = {eta}$", c=eta_dynamic_color(eta))

    ax.axhline(0, color="k", linestyle=":")
    ax.set_xlabel("wealth $x$")
    ax.set_ylabel("utility $u(x)$")
    ax.set_title("Isoelastic utility")
    ax.legend(loc=4)

In [ ]:
interact(
    plot_isoelastic_function,
    eta=widgets.FloatSlider(min=-1, max=1, value=1),
    x_min=widgets.fixed(0.01),
    x_max=widgets.fixed(100),
);

## Isoelastic wealth change

The main idea behind the wealth change based on utility function is to find a formula for a single step of deterministic wealth evolution with the following property: **corresponding change in utility is constant**, i.e., does not depend on time or initial wealth. The wealth change satisfying this property would lead to linear evolution of utility regardless of the number of applied changes or starting point. We  can express it as:
$$x(t+\Delta t)=u^{-1}(u(x(t))+\gamma\Delta t)\,$$
where $\gamma$ is a growth rate, and $u^{-1}$ is an inverse utility function. This equation directly express property of constant utility growth, because:
$$u(x(t+\Delta t))=uu^{-1}(u(x(t))+\gamma\Delta t)$$
$$u(x(t+\Delta t))=u(x(t))+\gamma\Delta t$$
$$\frac{u(x(t+\Delta t))-u(x(t))}{\Delta t}=\gamma$$
$$\lim_{\Delta t \rightarrow 0}\frac{u(x(t+\Delta t))-u(x(t))}{\Delta t}=\gamma$$
$$\frac{du}{dt}=\gamma$$
Wealth change constructed this way can be viewed as a three-step process:
1. transform wealth into utility
2. add $\Delta t\gamma$ utility
3. transform utility back to wealth

If we substitute Isoelastic utility function for $u(x)$ we get:
$$
x(t+\Delta t)=
\begin{cases}
(x^{1-\eta}+(1-\eta)\gamma\Delta t)^{\frac{1}{1-\eta}} & \text{for}\ \gamma\neq 1,\\
xe^{\gamma\Delta t} & \text{for}\ \gamma=1.\\
\end{cases}
$$

Let's find out in action how this wealth change works. The plot below consists of two subplots. The left subplot will show few steps of wealth evolution starting from the initial 1000 wealth units and applying the same wealth change multiple times. Sliders will control risk-aversion parameter for utility function, $\eta$, growth rate associated with wealth change, $\gamma$, and initial endowment $x_0$.   

> Notice that growth rates have different meanings depending on the dynamic because they correspond to utility changes and not wealth.

> This general class of wealth changes enables to recover three well-known dynamics: additive ($\eta=0$), multiplicative ($\eta=1$) and square-root ($\eta=0.5$). 

In [ ]:
def plot_wealth_and_utility_change(eta_dynamic, gamma, x0=1000, n_trials=5):
    # Initial setup
    tv = np.arange(n_trials)
    xv = np.zeros(n_trials)
    xv[0] = x0
    
    # Evolve wealth
    for t in tv[1:]:
        xv[t] = wealth_change(xv[t-1], gamma, eta_dynamic)  
    uv = isoelastic_utility(xv, eta_dynamic)

    # Visualize
    fig, (ax_x, ax_u) = plt.subplots(figsize=(15, 6), ncols=2)
    c = eta_dynamic_color(eta_dynamic)

    ax_x.plot(tv, xv, color=c, marker="o")
    for x in xv:
        ax_x.axhline(x, color=c, alpha=.3, ls=":")
    ax_x.set_ylabel("wealth $x(t)$")
    ax_x.set_xlabel("time $t$")
    ax_x.set_title("Changes in wealth")

    ax_u.plot(tv, uv, color=c, marker="o", label=f"$\eta = {eta_dynamic}$")
    for u in uv:
        ax_u.axhline(u, color=c, alpha=.3, ls=":")
    ax_u.set_ylabel("utility $u(x(t))$")
    ax_u.set_xlabel("time $t$")
    ax_u.set_title("Changes in utility")

    ax_u.legend(bbox_to_anchor=(1, 1))
    plt.tight_layout()

In [ ]:
eta_dynamic_widget = widgets.FloatSlider(min=-1, max=1, step=0.05, value=1)
gamma_widget = widgets.FloatSlider(min=-1, max=1, step=0.01, value=0.5)

def update_gamma_widget(*args):
    gamma_thr = 10 ** (3 - 3 * eta_dynamic_widget.value)
    gamma_widget.max = gamma_thr
    gamma_widget.min = -gamma_thr
    gamma_widget.value = 0.5 * gamma_thr
eta_dynamic_widget.observe(update_gamma_widget, 'value')

interact(
    plot_wealth_and_utility_change,
    eta_dynamic=eta_dynamic_widget,
    gamma=gamma_widget,
    x0=widgets.IntSlider(min=100, max=2000, step=100, value=1000),
    n_trials=widgets.fixed(5),
);

## Gamble dynamics

To get a better feel of how wealth can evolve under different dynamics, we will introduce the notion of a gamble. Here, **a gamble** will be a situation where an agent can get one of two outcomes with equal probabilities. Both outcomes represent isoelastic wealth changes with a fixed growth rate. We will only assume gambles composed of two wealth changes corresponding to the same dynamics. Hence, a gamble can be represented as a triplet, $(\gamma_1, \gamma_2, \eta)$, where $\gamma_1$ and $\gamma_2$ are growth rates for both wealth changes. 
> Note that for $\gamma_1=\gamma_2$ gamble becomes deterministic, i.e., only one outcome is possible. For $\gamma_1=\gamma_2=0$, we retrieve a null gamble, i.e., a gamble which does not affect wealth. For $\gamma_1=-\gamma_2$, a gamble has 0 expected change in wealth, i.e., average wealth will stay on the initial level after multiple repetitions. 

The plot below will show how an initial wealth can evolve after repeatedly applying the same gamble. Faded lines represent one of 300 individual wealth trajectories, whereas solid gray lines represent these individual trajectories' ensemble average. Sliders allow to control wealth dynamic $\eta$, both growth rates constituting a gamble, $\gamma_1$ and $\gamma_1$, and initial endowment $x_0$.

In [ ]:
def plot_coinflip_simulation(eta_dynamic, gamma_1, gamma_2, 
                             x0=1000, n_trials=100, n_sim=500):
    # Initial setup
    x = np.zeros((n_trials, n_sim))
    x[0] = x0
    
    # Randomly assign growth rates
    gammas = np.array([gamma_1, gamma_2])
    gammas_arr = gammas[np.random.randint(0, 2, size=(n_trials-1, n_sim))]

    # Evolve wealth
    for t in range(1, n_trials):
        x[t] = wealth_change(x[t-1], gamma=gammas_arr[t-1], eta=eta_dynamic)

    # Visualize
    fig, (ax, ax_hist) = plt.subplots(
        ncols=2, 
        gridspec_kw={'width_ratios': [4, 1]}, 
        figsize=(10, 6)
    )
    c = eta_dynamic_color(eta_dynamic)

    ax.plot(x, c=c, alpha=0.1, marker=".", mfc="k");
    ax.plot(np.mean(x, axis=1), c="k", alpha=0.75, lw=3);
    ax.set_xlim([0, len(x) - 1])
    ax.set_xlabel("time $t$")
    ax.set_ylabel("wealth $x(t)$")

    ax_hist.hist(x[-1], bins=30, orientation="horizontal", color="k", alpha=0.25)
    ax_hist.axhline(np.mean(x[-1]), c="k", alpha=0.75, lw=3)
    ax_hist.set_xlabel("counts")
    ax_hist.set_ylim(ax.get_ylim())
    ax_hist.set_yticks([])

    plt.tight_layout()

In [ ]:
eta_dynamic_coinflip_widget = widgets.FloatSlider(min=-1, max=1, step=0.05, value=1)
gamma_1_widget = widgets.FloatSlider(min=-1, max=1, step=0.01, value=0.1)
gamma_2_widget = widgets.FloatSlider(min=-1, max=1, step=0.01, value=-0.08)

def update_gammas_widget(*args):
    gamma_thr = 10 ** (3 - 3 * eta_dynamic_coinflip_widget.value)
    for widget in [gamma_1_widget, gamma_2_widget]:
        widget.max = gamma_thr
        widget.min = -gamma_thr
    gamma_1_widget.value = 0.1 * gamma_thr
    gamma_2_widget.value = -0.08 * gamma_thr
    
eta_dynamic_coinflip_widget.observe(update_gammas_widget, 'value')

interact(
    plot_coinflip_simulation,
    eta_dynamic=eta_dynamic_coinflip_widget,
    gamma_1=gamma_1_widget,
    gamma_2=gamma_2_widget,
    x0=widgets.IntSlider(min=100, max=2000, step=100, value=1000),
    n_trials=widgets.fixed(100),
    n_sim=widgets.fixed(250)
);